# Vertex Updating
This notebook won't use predicted normals to update the vertices to a correct position, but generates noise and tries to update the vertex positions with the ground truth normals to see if the vertex updating method works.

First we create a noisy mesh.
Then we extract the needed information.
Then we create and apply the vertex updating algorithm.
Finally, we show what the algorithm does and whether it works.

In [20]:
from PatchGeneration.Modules.Mesh import Mesh
import meshplot as mp
import numpy as np
import copy
import time

# Generate noisy mesh

In [21]:
myMesh = Mesh.readFile("PatchGeneration/Object/example_object.obj")
myMesh.applyGaussianNoise(0.3)
mp.plot(myMesh.getVertices(), myMesh.f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0995235…

# Extract information

In [22]:
# Iterations of the algorithm
k = 15
# Original normals of the mesh
n = myMesh.getFaceNormals(False)
# Vertices
v = copy.deepcopy(myMesh.getVertices())
v2 = copy.deepcopy(v)
v3 = copy.deepcopy(v)
# Faces of mesh
f = myMesh.f
# Vertex Triangle Adjacency
vta = myMesh.getVertexTriangleAdjacency()

# Algorithm 1

In [23]:
start = time.time()
# Repeat algorithm k times
for ki in range(k):
    print(f"Loop {ki+1}/{k}")
    # Loop over vertices
    for vi in range(len(v)):
        # Loop over faces
        faces = vta[0][np.arange(vta[1][vi], vta[1][vi+1])]
        S = np.zeros((3,))
        for fj in faces:
            # Loop over edges
            # vks = (3,) = (Vertex indices of face fj)
            vks = f[fj]
            # dvs = (3, 3) = (Vertex index, Position axis (x, y, z))
            dvs = v[vks] - v[vi] # One of these is gonna be vi - vi = 0
            # nj = (3,) = (Position axis (x, y, z))
            nj = n[fj]
            # dot = {nj -> (repeated 3x, nj Position axis) -> sum over axis 1 (position axis that got multiplied)} = dot product of 3 vertices
            dot = np.sum(np.tile(nj, (3, 1)) * dvs, axis=1)
            # elements are the elements to be summed for this face.
            # elements = (dot product of vertex index k, scaled normal vector j)
            elements = nj.reshape(3, 1) @ dot.reshape(1, 3)
            # Sum 3 results and add it to the total sum (it needs to be summed anyways)
            S += np.sum(elements, axis=0)
        # Divide sum by number of neighbouring faces and add the resulting offset to the vertex.
        scaled_S = S / (3 * len(faces))
        v[vi] += scaled_S
end = time.time()
print(f"Algorithm took {int(end - start)} seconds with an average of {(end - start)/k} seconds per loop!")
ov = myMesh.getVertices(False)
d = ov - v
vMSE = np.mean(np.square(np.linalg.norm(d, axis=1)))
print(f"MSE for vertex distances: {vMSE}")
# (f, v, ax)
fv = v[f]
# (f, ax)
crosses = np.cross(fv[:, 1, :] - fv[:, 0, :], fv[:, 2, :] - fv[:, 1, :])
# (f, ax)
n1 = crosses / np.linalg.norm(crosses, axis=1)[:, None]
# (f,)
dot_n = np.sum(n1 * n, axis=1)
nMSE = 1 - np.mean(np.square(dot_n))
print(f"MSE for normal angle is {nMSE}")

Loop 1/15
Loop 2/15
Loop 3/15
Loop 4/15
Loop 5/15
Loop 6/15
Loop 7/15
Loop 8/15
Loop 9/15
Loop 10/15
Loop 11/15
Loop 12/15
Loop 13/15
Loop 14/15
Loop 15/15
Algorithm took 61 seconds with an average of 4.1164462248484295 seconds per loop!
MSE for vertex distances: 0.15080020748846754
MSE for normal angle is 0.049792458959195596


# Algorithm 2

In [24]:
start = time.time()
# Repeat algorithm k times
for ki in range(k):
    print(f"Loop {ki+1}/{k}")
    # Loop over vertices
    for vi in range(len(v2)):
        # Loop over faces
        # (f,)
        faces = vta[0][np.arange(vta[1][vi], vta[1][vi+1])]
        # Loop over edges
        # vks = (f, 3)
        vks = f[faces]
        # dvs = (f, 3, 3) = (face index, vertex index, axis index)
        dvs = v2[vks] - v2[vi] # One of these is gonna be vi - vi = 0
        # nj = (f, 3) = (face index, axis index)
        nj = n[faces]
        # dot = nj -> (f, 3) -> (f, 3, 1) -> (f, 3, 3) -> (face index, empty, axis index) * (face index, vertex index, axis index) -> sum over axis index (axis 2) -> dot product: (face index, vertex index)
        dot = np.sum(np.tile(nj[:, None], (1, 3, 1)) * dvs, axis=2)
        # elements are the elements to be summed for this face.
        # elements = [dot product: (face index, vertex index, empty)] * [normal vector: (face index, empty, axis index)] = (face index, vertex index, axis index)
        elements = dot[..., None] * nj[:, None]
        S = np.sum(np.sum(elements, axis=0), axis=0)
        # Divide sum by number of neighbouring faces and add the resulting offset to the vertex.
        scaled_S = S / (3 * len(faces))
        v2[vi] += scaled_S
end = time.time()
print(f"Algorithm took {int(end - start)} seconds with an average of {(end - start)/k} seconds per loop!")
ov = myMesh.getVertices(False)
d = ov - v2
vMSE = np.mean(np.square(np.linalg.norm(d, axis=1)))
print(f"MSE for vertex distances: {vMSE}")
# (f, v, ax)
fv = v2[f]
# (f, ax)
crosses = np.cross(fv[:, 1, :] - fv[:, 0, :], fv[:, 2, :] - fv[:, 1, :])
# (f, ax)
n2 = crosses / np.linalg.norm(crosses, axis=1)[:, None]
# (f,)
dot_n = np.sum(n2 * n, axis=1)
nMSE = 1 - np.mean(np.square(dot_n))
print(f"MSE for normal angle is {nMSE}")


Loop 1/15
Loop 2/15
Loop 3/15
Loop 4/15
Loop 5/15
Loop 6/15
Loop 7/15
Loop 8/15
Loop 9/15
Loop 10/15
Loop 11/15
Loop 12/15
Loop 13/15
Loop 14/15
Loop 15/15
Algorithm took 15 seconds with an average of 1.0642414887746174 seconds per loop!
MSE for vertex distances: 0.042943124576087656
MSE for normal angle is 0.0007286887536466224


# Algorithm 3

In [25]:
start = time.time()
# Repeat algorithm k times
for ki in range(k):
    print(f"Loop {ki+1}/{k}")
    # Loop over vertices
    _V = len(v3)
    # (vi,)
    vis = np.arange(_V)
    # Loop over faces
    starts = np.delete(vta[1], -1)
    stops = np.delete(vta[1], 0)
    degrees = stops - starts
    MAX_DEGREE = np.max(degrees)
    # 1D array of 1D array ranges
    ranges = np.frompyfunc(np.arange, 2, 1)(starts, stops)
    # ranges (vi, fm) where fm means max degree of all vertices
    ranges_padded = np.stack([np.pad(r, (0, MAX_DEGREE - len(r)), 'constant', constant_values=(-1, -1)) for r in ranges])
    mask_is_padded = ranges_padded == -1
    # faces: (vi, fm)
    faces = vta[0][ranges_padded]
    # Loop over edges
    # vks = (vi, fm, vj)
    vks = f[faces]
    # dvs = (vi, fm, vj, ax)
    dvs = v3[vks] - np.tile(v3[vis][:, None, None, :], (1, MAX_DEGREE, 3, 1)) # One of these is gonna be vi - vi = 0
    # nj = (vi, fm, ax)
    nj = n[faces]
    # dot = (vi, fm, vj) = (vi, fm, empty, ax) * (vi, fm, vj, ax)
    dot = np.sum(np.tile(nj[:, :, None, :], (1, 1, 3, 1)) * dvs, axis=3)
    # elements are the elements to be summed for this face.
    # elements = (vi, fm, vj, ax) = (vi, fm, vj, empty) * (vi, fm, empty, ax)
    elements = dot[..., None] * nj[:, :, None]
    # Set padded entries back to 0, such that they don't influence vertex positions
    elements[mask_is_padded] = 0
    # Sum over fm and vj, S = (vi, ax)
    S = np.sum(np.sum(elements, axis=1), axis=1)
    # Divide sum by number of neighbouring faces and add the resulting offset to the vertex.
    # Scaled (vi, ax) = S (vi, ax) / degrees (vi,)
    scaled_S = S / (3 * np.tile(degrees[:, None], (1, 3)))
    v3 += scaled_S
end = time.time()
print(f"Algorithm took {int(end - start)} seconds with an average of {(end - start)/k} seconds per loop!")
ov = myMesh.getVertices(False)
d = ov - v3
vMSE = np.mean(np.square(np.linalg.norm(d, axis=1)))
print(f"MSE for vertex distances: {vMSE}")
# (f, v, ax)
fv = v3[f]
# (f, ax)
crosses = np.cross(fv[:, 1, :] - fv[:, 0, :], fv[:, 2, :] - fv[:, 1, :])
# (f, ax)
n3 = crosses / np.linalg.norm(crosses, axis=1)[:, None]
# (f,)
dot_n = np.sum(n3 * n, axis=1)
nMSE = 1 - np.mean(np.square(dot_n))
print(f"MSE for normal angle is {nMSE}")

Loop 1/15
Loop 2/15
Loop 3/15
Loop 4/15
Loop 5/15
Loop 6/15
Loop 7/15
Loop 8/15
Loop 9/15
Loop 10/15
Loop 11/15
Loop 12/15
Loop 13/15
Loop 14/15
Loop 15/15
Algorithm took 8 seconds with an average of 0.5907633463541667 seconds per loop!
MSE for vertex distances: 0.043037224090930484
MSE for normal angle is 0.0009190578806473892


# Visualize result

In [26]:
v_plot = copy.deepcopy(v)
v2_plot = copy.deepcopy(v2)
v3_plot = copy.deepcopy(v3)
v_plot[:, 0] += myMesh.getPCBoundingBox()[0]
v2_plot[:, 0] += 2 * myMesh.getPCBoundingBox()[0]
v3_plot[:, 0] += 3 * myMesh.getPCBoundingBox()[0]
v_plot = np.append(copy.deepcopy(myMesh.getVertices(True)), v_plot, axis=0)
v_plot = np.append(v_plot, v2_plot, axis=0)
v_plot = np.append(v_plot, v3_plot, axis=0)
f_plot = np.tile(f, (4, 1))
f_plot[len(f):2*len(f)] += len(v)
f_plot[2*len(f):3*len(f)] += 2*len(v)
f_plot[3*len(f):] += 3*len(v)
mp.plot(v_plot, f_plot)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(98.001981…

In [27]:
print(np.allclose(myMesh.getVertices(False), v2))
mp.plot(v2, f)

False


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.011758…

2.938901805109901e-06
